Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

In [4]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [5]:
def date_sorter():
    import re
    import datetime
    f11 = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})\b')
    f12 = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{2})\b')
    f12[2] = '19' + f12[2]
    f1 = pd.concat([f11,f12])
    f1.reset_index(inplace = True)
    f1_index = f1['level_0']
    f2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.,]*) (\d{2}[,-]*) (\d{4})')
    f2.reset_index(inplace = True)
    f2_index = f2['level_0']
    f35 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:.,-]* )(\d{4})')
    f35.reset_index(inplace = True)
    f35_index = f35['level_0']
    f6 = df.str.extractall(r'((?:\d{1,2}))(/)(\d{4})') 
    f6.reset_index(inplace = True)
    f6_index = f6['level_0']
    rn = []
    for i in f6_index:
        if not (i in f1_index.values):
            rn.append(i)
    f6 = f6[f6['level_0'].isin(rn)]
    f71= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    f72 = df.str.extractall(r'^(\d{4})[^0-9]')
    f7 = pd.concat([f71,f72])
    f7.reset_index(inplace = True)
    f7_index = f7['level_0']
    rn1 = []
    for i in f7_index:
        if not ((i in f6_index.values)|(i in f2_index.values)|(i in f35_index.values)):
                   rn1.append(i)
    f7 = f7[f7['level_0'].isin(rn1)]
    f1['new'] = f1[0] + '/' + f1[1] + '/' + f1[2] 
    f1['new1'] = pd.to_datetime(f1['new'], format = '%m/%d/%Y') #f1 format
    mon = []
    dat = []
    for i in f2[0]:
        mon.append(i[:3])
    mon = pd.Series(mon)
    f2['mon'] = mon
    for i in f2[1]:
        dat.append(i[:2])
    dat = pd.Series(dat)
    f2['dat'] = dat
    f2['new'] = f2['dat'] + f2['mon'] + f2[2]
    fin = []
    for i in range(len(f2)):
        fin.append(datetime.datetime.strptime(f2['new'][i],'%d%b%Y'))
    fin = pd.Series(fin)
    f2['new1'] = fin
    f2#final format
    f35[0] = f35[0].fillna('1 ')
    mon1 = []
    for i in f35[1]:
        mon1.append(i[:3])
    mon1 = pd.Series(mon1)
    f35['mon1'] = mon1
    f35['new'] = f35[0]+f35['mon1']+f35[2]
    fin1 = []
    for i in range(len(f35)):
        fin1.append(datetime.datetime.strptime(f35['new'][i],'%d %b%Y'))
    fin1 = pd.Series(fin1)
    f35['new1'] = fin1 #finalformal
    f6['new'] = f6[0] + '/' + '1' + '/' + f6[2]
    f6['new1'] = pd.to_datetime(f6['new'], format = '%m/%d/%Y')  #finalformal f6
    f7['new'] = '1'+'/'+'1'+'/'+f7[0]
    f7['new1'] = pd.to_datetime(f7['new'], format = '%m/%d/%Y')
    frame = [f1[['level_0','new1']], f2[['level_0','new1']], f35[['level_0','new1']], f6[['level_0','new1']], f7[['level_0','new1']]]
    final = pd.concat(frame)
    final.sort_values('new1', inplace = True)
    final.reset_index(inplace = True)
    final['level_0']

        # Your code here
    
    return final['level_0']
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    480
477    286
478    244
479    431
480    279
481    198
482    463
483    381
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64